### Load tensorflow

In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

### Read training and test data

In [0]:
import pandas as pd
train_df = pd.read_csv('data/flower_photos/train.csv')
test_df = pd.read_csv('data/flower_photos/test.csv')

In [3]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Flower classes: ', class_names)

Flower classes:  ['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses']


In [0]:
#Define some parameters
img_size = 224
img_depth = 3  

### Build batch generator

In [0]:
import numpy as np

In [0]:
def batch_generator(df, batchsize=32, train_mode=True):   
    
    img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                   width_shift_range=0.2,
                                                                   height_shift_range=0.2,
                                                                   horizontal_flip=True)
    
    while True:       
        
        
        #Generate random numbers to pick images from dataset
        batch_nums = np.random.randint(0,df.shape[0],batchsize)
        
        #Initialize batch images array
        batch_images = np.zeros((batchsize,img_size, img_size,img_depth))
        
        #Initiate batch label array
        batch_labels = np.zeros((batchsize, len(class_names)))
        
        for i in range(batchsize):
            
            #Load image
            flower_image =  tf.keras.preprocessing.image.load_img('data/' + 
                                                                  df.loc[batch_nums[i]]['image_file'],
                                                                  target_size=(img_size, img_size))
            #Convert to array
            flower_image = tf.keras.preprocessing.image.img_to_array(flower_image)
            
            if(train_mode):
                #Apply transform
                flower_image =  img_generator.random_transform(flower_image)
            
            
            #Get the class
            img_class = df.loc[batch_nums[i]]['class']
            #Conver class to number
            img_class = class_names.index(img_class)
            #Convert class to one hot encoding
            img_class = tf.keras.utils.to_categorical(img_class, num_classes=len(class_names))
            
            #Update batch images and class arrays
            batch_images[i] = tf.keras.applications.resnet50.preprocess_input(flower_image)
            batch_labels[i] = img_class
        
        yield batch_images, batch_labels          

### Load pre-trained model

In [7]:
model = tf.keras.applications.resnet50.ResNet50(include_top=False, #Do not include FC layer at the end
                                          input_shape=(img_size,img_size, img_depth),
                                          weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [8]:
model.output

<tf.Tensor 'activation_48/Relu:0' shape=(?, 7, 7, 2048) dtype=float32>

In [0]:
#Set pre-trained model layers to not trainable
for layer in model.layers:
    layer.trainable = False

### Add FC layer for new classes

In [0]:
#get Output layer of Pre0trained model
x = model.output

#Flatten the output to feed to Dense layer
x = tf.keras.layers.Flatten()(x)

#Add one Dense layer
x = tf.keras.layers.Dense(200, activation='relu')(x)

#Add output layer
prediction = tf.keras.layers.Dense(len(class_names),activation='softmax')(x)

### Building final model for Classification

In [0]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [0]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
final_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

### Train the model

In [0]:
#Create train and test generator
batchsize = 64
train_generator = batch_generator(train_df, batchsize=batchsize) #batchsize can be changed
test_generator = batch_generator(test_df, batchsize=batchsize, train_mode=False)

In [15]:
final_model.fit_generator(train_generator, 
                          epochs=30,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
45/45 [==============================] - 63s 1s/step - loss: 8.1466 - acc: 0.4736 - val_loss: 8.0679 - val_acc: 0.4957
Epoch 2/30
45/45 [==============================] - 60s 1s/step - loss: 7.3721 - acc: 0.5382 - val_loss: 6.5091 - val_acc: 0.5852
Epoch 3/30
45/45 [==============================] - 56s 1s/step - loss: 5.8607 - acc: 0.6292 - val_loss: 6.4703 - val_acc: 0.5938
Epoch 4/30
45/45 [==============================] - 54s 1s/step - loss: 5.2567 - acc: 0.6677 - val_loss: 5.9079 - val_acc: 0.6293
Epoch 5/30
45/45 [==============================] - 59s 1s/step - loss: 4.8865 - acc: 0.6910 - val_loss: 6.4877 - val_acc: 0.5966
Epoch 6/30
45/45 [==============================] - 60s 1s/step - loss: 5.5911 - acc: 0.6497 - val_loss: 5.8403 - val_acc: 0.6293
Epoch 7/30
45/45 [==============================] - 56s 1s/step - loss: 5.3936 - acc: 0.6618 - val_loss: 5.2570 - val_acc: 0.6733
Epoch 8/30
45/45 [========================

In [16]:
print(len(model.layers))
for layer in model.layers[150:]:
    layer.trainable =  True    

175


In [0]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
final_model.fit_generator(train_generator, 
                          epochs=30,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize)

Epoch 1/30
45/45 [==============================] - 65s 1s/step - loss: 6.5796 - acc: 0.5882 - val_loss: 12.2717 - val_acc: 0.2386
Epoch 2/30
45/45 [==============================] - 61s 1s/step - loss: 5.5709 - acc: 0.6507 - val_loss: 11.6108 - val_acc: 0.2784
Epoch 3/30
45/45 [==============================] - 56s 1s/step - loss: 5.1908 - acc: 0.6750 - val_loss: 7.3508 - val_acc: 0.5426
Epoch 4/30
45/45 [==============================] - 56s 1s/step - loss: 4.7578 - acc: 0.7024 - val_loss: 6.1216 - val_acc: 0.6179
Epoch 5/30
45/45 [==============================] - 58s 1s/step - loss: 4.4892 - acc: 0.7177 - val_loss: 5.2861 - val_acc: 0.6705
Epoch 6/30
45/45 [==============================] - 62s 1s/step - loss: 5.2946 - acc: 0.6681 - val_loss: 7.5559 - val_acc: 0.5241
Epoch 7/30
45/45 [==============================] - 62s 1s/step - loss: 5.4568 - acc: 0.6590 - val_loss: 6.8694 - val_acc: 0.5682
Epoch 8/30
45/45 [==============================] - 60s 1s/step - loss: 4.8279 - acc: 0.